# Feature Maps for Polynomial and Anova Kernels
*@author: Chen Wang* <br />
*Email: jhwjhw0123@gmail.com* <br />
*Feel free to copy or use, just remember to mention the source.* <br />
# Brief Introduction
This program designed a general function to compute the feature map for Polynomial kernels and Anova kernels. The general form of the Polynomial kernel is: <br />
$$K_{p}(\boldsymbol{x},\boldsymbol{y})=(\boldsymbol{x}^{T}\boldsymbol{y}+1)^{d}$$
Where the general form of Anova kernel should be: <br />
$$K_{a}(\boldsymbol{x},\boldsymbol{y})=\prod_{i=1}^{n}(x_{i}y_{i}+1)^{d}$$
Both of these two kind of kernels have explicit feature maps to be represented. Now we will discuss them respectively.<br />
## Polynomial Kernel
The feature map of the $d$-order polynomial kernel would be complex, it consists of the terms with: <br />
$$\phi(\boldsymbol{x})=\{\sqrt{\binom{d}{i_1,i_2,...,i_n}}x_{1}^{i_1}x_{2}^{i_2}...x_{n}^{i_n}\}$$
And the terms should be consisted of: <br />
$$\sum_{j=0}^{n}i_{j}=\{1,2,3,...,d\}$$
The above equation means that for each order no more than $d$, we would be needing to compute all possible combimnations that *sum to the order*. For example, if the current $d^{*}$ is $d^{*}=2$ and $n=3$, we will need to have $\{x_{1}x_{2},x_{1}x_{3},x_{2}x_{3},x_{1}^{2},x_{2}^{2},x_{3}^{2}\}$. And keep reminding yourself that this is for only one of all the $d$ values! Nevertheless, with a iteration recipe (function call itself), here we achieve the goal by the following method:<br />
1. For each order $d^{*}$ from $1$ to $d$
2. Compute the coefficient $\sqrt{\binom{d}{d^{*}}}$
   And then, 
   For each 'depth' (the highest order of each component) from $d^{*}$ to $1$
3. For each component of the $\boldsymbol{x}$ $x_{1}$,$x_{2}$,...$x_{n}$
4. Get a copy of $\boldsymbol{x}$
5. Pick the current component *from the copy* and store $c = x_{i}$
6. If current order $d^{**}$ = $d^{*}$:
    then compute the result $r = c*{x_{j}}^{d^{**}}$<br />
    else:
    then $c = c*x_{j}$, and then set $d^{**} = d^{**} - 1$, call the function again with $\boldsymbol{x}_{*} = \boldsymbol{x}/x_{i}x_{j}$
7. After finish the iteration started by 3, delete the component $x_{i}$ from the copy to *avoid repeative cimputation*, but keep the original one to keep the iteration available.
8. After iterating over 2 and 1, return the result <br />

For more information one could inspect the comments in the code. <br />
## Anova Kernel
The feature map of Anova kernel is simpler, perhaps that's the reason it sometimes more preferable than the polynomial kernel. The feature map of Anova kernel is: <br />
$$\phi(\boldsymbol{x})=\{x_{1},x_{2},\cdots,x_{n},x_{1}x_{2},\cdots,x_{1}x_{n},\cdots,x_{1}x_{2}...x_{n}\}$$
So we could see although we still need to consider the terms in every oder that sum up to $d_{*}$, we only need to consider the 'oder-1 multiplication' at each order. So given the fact that we've already got the method of polynomial kernel, we could do this easily by ignoring step 6 of the previous routine, and simply set 'depth = 1' for every order. <br />


# **Now let's see the codes!**

In [1]:
#Import essential packages
import random
import numpy as np
from numpy import linalg
from scipy.misc import comb
import matplotlib.pyplot as plt

In [2]:
# The function which could be itarately called
# -- This is a little bit complex, see the introduction part if you want explaination
def iterate_feature_compute(data_com,input_array,order_input,order_depth,initial_flag=False):
    '''
    :param data_com: The input data, 'com' means combination. The data might be part of the original or whole
    :param input_array: this is the array to keep the multiplication information, it would be automatically adjusted
                        to 'ones' array if it is the initial step, however it is still a good habit to input a [nData*1]
                        ones matrix at the original
    :param order_input: The polynomial order d of the feature map/kernel, serving as the highest order of the map
    :param order_depth: Compute which extent of the features of the kernel. It should be (1 ~ order_input), where 1 stands
                        for all the components has a highest order of 1, and order_input stands for a set of high-oder
                        features that don't combine with others
    :param initial_flag: The flag to indicate if the function is in the starting step. This is added because the routine
                        is designed as an iteration, and since we need 'input_array' to be 'ones' matrix at the beginning,
                        this parameter could help use avoid exceptions
    :return: The features of the given order and depth
    '''
    if initial_flag==True:
        input_array = np.ones([data_com.shape[0],1])
        initial_flag = False
    else:
        pass
    if order_input == order_depth:
        # If it should be high order features then we compute the power operation
        data_com_power = np.power(data_com,order_input)
        # multiply with the input array, which stands for the stored multiplication information
        # (for example, if we need [x_1 * x_2 * x_3^2], then the information of [x_1 * x_2] will be in the input array)
        return_array = np.multiply(np.matlib.repmat(input_array,1,data_com_power.shape[1]),data_com_power)
    else:
        # initialize the return array
        data_com_copy = np.array(data_com)
        return_array = np.zeros((data_com.shape[0],0))
        nDim = data_com.shape[1]
        for dim_var in range(nDim):
            # delete the
            new_input_data = np.delete(data_com_copy,dim_var,axis=1)
            # delete all the columns with nan value
            new_input_data = new_input_data[:,~np.all(np.isnan(new_input_data), axis=0)]
            if new_input_data.shape[1] == 0:
                break
            # store the multiplication information
            further_input = np.multiply(input_array,np.reshape(data_com[:,dim_var],[data_com.shape[0],1]))
            # iteratively call the function to compute
            new_features = iterate_feature_compute(new_input_data,further_input,order_input-1,order_depth)
            # append new feature to the return form
            return_array = np.concatenate((return_array,new_features),axis=1)
            # This is the code to create NaN value, the purpose of this is to avoid redundant combinations. We could see
            # in the above routine we deleted the columns with nan values
            data_com_copy[:,dim_var] = np.nan

    return return_array

In [3]:
# Function to compute the feature map terms of order d: this is only used for Polynomial one -- in Anova one we could
# direcly call the above function
def order_n_feature_compute(data,order_this,order_total):
    '''
    :param data: the iput data
    :param order: the desired of order of computation
    :return: The features of order d as the feature map of polynomial kernel
    '''
    this_order_coefficient = np.sqrt(comb(order_total, order_this))
    nData = data.shape[0]

    initial_input_array = np.ones([nData,1])
    feature_array = np.zeros([nData,0])

    for c_oder in range(order_this,0,-1):
        this_feature_array = iterate_feature_compute(data,initial_input_array,order_input=order_this,
                                                     order_depth=c_oder,initial_flag=True)
        feature_array = np.concatenate((feature_array,this_feature_array),axis=1)

    feature_array = this_order_coefficient*feature_array

    return feature_array

Now we could finally write our polynomial kernel routine...

In [4]:
def polynomial_feature_map(data,order):

    assert type(data) == type(np.array([1]))

    nData = data.shape[0]
    nDim = data.shape[1]

    polynomial_feature_matrix = np.ones([nData,1])  # 'Ones' would serve as the bias term
    for c_order in range(order):
        this_order_input = c_order+1
        this_order_feature = order_n_feature_compute(data,this_order_input,order)
        polynomial_feature_matrix = np.concatenate((polynomial_feature_matrix,this_order_feature),axis=1)

    return polynomial_feature_matrix

Seems not very hard! Actually, if you find it's too tricky in the previous two functions, I think it would be not hard to understand this polynomial feature map routine. If you are keen then try to comprehend it! <br />
And then we could have the Anova feature map:

In [5]:
def Anova_feature_map(data,order):
    '''
    Anova feature map is generally more straightforward than the polynomial one. It only demands [x1,x2,...,xn,...x1x2xn]
    and we don't need to provide the intermediate terms. In our program, this is equivalent to setting all the 'depth' into
    the 'full order'
    :param data: the original data
    :param order: the order of the ANOVA we demand
    :return: the features of order d feature map of ANOVA kernel
    '''

    assert type(data) == type(np.array([1]))

    nData = data.shape[0]
    nDim = data.shape[1]

    Anova_feature_matrix = np.ones([nData,1])  # 'Ones' would serve as the bias term
    initial_input_array = np.ones([nData,1])
    for c_order in range(order):
        this_order_input = c_order + 1
        this_order_feature = iterate_feature_compute(data, initial_input_array, order_input=this_order_input,
                                                     order_depth=this_order_input, initial_flag=True)
        Anova_feature_matrix = np.concatenate((Anova_feature_matrix, this_order_feature), axis=1)

    return Anova_feature_matrix

And here is a demonstration of the above code: 

In [6]:
data = np.array([[1,2,3],[11,22,33]],dtype='float')  
poly_feature_data = polynomial_feature_map(data,3)
anova_feature_data = Anova_feature_map(data,3)

In [7]:
print(poly_feature_data)

[[  1.00000000e+00   1.73205081e+00   3.46410162e+00   5.19615242e+00
    1.73205081e+00   6.92820323e+00   1.55884573e+01   3.46410162e+00
    5.19615242e+00   1.03923048e+01   1.00000000e+00   8.00000000e+00
    2.70000000e+01   4.00000000e+00   9.00000000e+00   1.80000000e+01
    6.00000000e+00]
 [  1.00000000e+00   1.90525589e+01   3.81051178e+01   5.71576766e+01
    2.09578148e+02   8.38312591e+02   1.88620333e+03   4.19156295e+02
    6.28734443e+02   1.25746889e+03   1.33100000e+03   1.06480000e+04
    3.59370000e+04   5.32400000e+03   1.19790000e+04   2.39580000e+04
    7.98600000e+03]]


In [8]:
print(anova_feature_data)

[[  1.00000000e+00   1.00000000e+00   2.00000000e+00   3.00000000e+00
    1.00000000e+00   4.00000000e+00   9.00000000e+00   1.00000000e+00
    8.00000000e+00   2.70000000e+01]
 [  1.00000000e+00   1.10000000e+01   2.20000000e+01   3.30000000e+01
    1.21000000e+02   4.84000000e+02   1.08900000e+03   1.33100000e+03
    1.06480000e+04   3.59370000e+04]]


Here you could find I use 'float' type -- why? Well, if you go through the codes there is a part to set the value of NaN. Actually this is the part that I 'delete the used component' (see the step 7). Using NaN is more robust because it prevent the data from changing size and cause unpredicteable problem in for loop. However, if NaN goes to integer (or other type) then it could be set to a (strange) number. This is not very good -- we need to keep the data type as float to compute. 

And that's it, the general $d$-order feature map for polynomial and Anova kernel. 